# Plot results for predicting hurricane strength (maximum intensity).
author: Elizabeth A. Barnes and Randal J. Barnes

version: v2.0

date: 28 October 2021

In [1]:
import datetime
import os
import pickle
import pprint
import time

import matplotlib as mpl
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


import experiment_settings
from build_data import build_hurricane_data
import shash
from toolbox import custom_round
import prediction

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [2]:
__author__ = "Randal J Barnes and Elizabeth A. Barnes"
__version__ = "28 October 2021"

# EXP_NAME = "intensity101_ALEPCP72"
# EXP_NAME = "intensity8_EPCP48"
EXP_NAME = "intensity7_AL72"
# EXP_NAME = "intensity5_AL72"
# EXP_NAME = "intensity6_EPCP48"


DATA_PATH = "data/"
MODEL_PATH = "saved_models/"

In [3]:
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
dpiFig = 300.
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

settings = experiment_settings.get_settings(EXP_NAME)
pprint.pprint(settings, width=80)

{'act_fun': 'relu',
 'basin': 'AL',
 'batch_size': 64,
 'distribution': 'shash3',
 'filename': 'nnfit_vlist_intensity_and_track_extended.dat',
 'hiddens': [15, 10],
 'leadtime': 72,
 'learning_rate': 0.0001,
 'loss': 'likelihood',
 'momentum': 0.9,
 'n_epochs': 25000,
 'n_test': 0,
 'n_val': 300,
 'nesterov': True,
 'np_seed': 890,
 'patience': 300,
 'ridge_param': 0.0,
 'target': 'intensity',
 'undersample': False}


## Build the intensity data tensors.

In [4]:
for np_seed in (888,889,890):
    
    print('------------------')
    print('NP_SEED = ' + str(np_seed))
    
    np.random.seed(np_seed)  
    (
        x_train,
        onehot_train,
        x_val,
        onehot_val,
        data_summary,
        df_val,
    ) = build_hurricane_data(DATA_PATH, settings, verbose=0)
    
    MODEL_NAME   = EXP_NAME + "_" + f"network_seed_{np_seed}_numpy_seed_{np_seed}"
    model        = tf.keras.models.load_model(MODEL_PATH + MODEL_NAME + "_model", compile=False) 
    
    # make predictions
    mu_pred, sigma_pred, gamma_pred, tau_pred = prediction.params(x_val, model)    

    # convert y_pred to intensity corrections    
    ann_correction  = prediction.percentile_value(mu_pred,sigma_pred,gamma_pred,tau_pred,percentile_frac=.5)
    true_correction = df_val['OBDV'].to_numpy()  

    # convert to full intensity predictions
    cons_intensity  = df_val['VMXC'].to_numpy()
    ann_intensity   = cons_intensity + ann_correction
    true_intensity  = cons_intensity + true_correction

    # covert ann prediction bounds
    ann_low_correction  = prediction.percentile_value(mu_pred,sigma_pred,gamma_pred,tau_pred,percentile_frac=.25)
    ann_high_correction = prediction.percentile_value(mu_pred,sigma_pred,gamma_pred,tau_pred,percentile_frac=.75)
    iqr                 = ann_high_correction - ann_low_correction

    ann_low_intensity   = ann_low_correction + cons_intensity
    ann_high_intensity  = ann_high_correction + cons_intensity

    # compute errors
    ann_error  = np.abs(ann_intensity - true_intensity)
    cons_error = np.abs(cons_intensity - true_intensity)

    ann_mae    = np.mean(ann_error)
    cons_mae   = np.mean(cons_error)

    print('ann  MAE = ' + str(np.round(ann_mae,3)))
    print('cons MAE = ' + str(np.round(cons_mae,3)))          
    print('percent improvement by ANN = ' + str(np.round(100.*(cons_mae-ann_mae)/cons_mae,2)) + '%')    

------------------
NP_SEED = 888
ann  MAE = 11.232
cons MAE = 12.403
percent improvement by ANN = 9.44%
------------------
NP_SEED = 889
ann  MAE = 11.294
cons MAE = 12.339
percent improvement by ANN = 8.47%
------------------
NP_SEED = 890
ann  MAE = 10.965
cons MAE = 12.027
percent improvement by ANN = 8.83%
